In [2]:
## Imports
import numpy as np
import tensorflow as tf
import time
import os
import re

from tensorflow import keras
from tensorflow.keras.models import Sequential#, Model
from tensorflow.keras.layers import Dense, Dropout#, Flatten, Input, Activation, Bidirectional
from tensorflow.keras.layers import TimeDistributed
#from tensorflow.keras.layers import Lambda, concatenate
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN#, RNN, Conv1D, MaxPooling1D

from tensorflow.keras.optimizers import Adam#, SGD, RMSprop, Nadam
from tensorflow.keras import backend as K
#from tensorflow.keras import regularizers

# from sklearn.preprocessing import MinMaxScaler
#from sklearn.metrics import *

#import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
# import pandas_datareader as pdr
# import yfinance as yf
# import quandl

from keras.models import load_model
# from joblib import load
import json

### Game Rules
When it's your turn to play, you must pick a letter. This letter added to the previous letters cannot form a word or you lose. At the same time, there has to be a word that starts with the letters played.

In [3]:

import bisect

def get_lexicon(language):
    '''
    Defines the lexicon and alphabet to be used.
    Arguments:
        String - language: Language of lexicon
    Returns:
        lex - List containing the words in the specified lexicon
        alphabet - String containing all acceptable letters in alphabet
    '''
    if language=='SE':
        filename = 'swedish_lexicon.txt'
        alphabet = 'abcdefghijklmnopqrstuvwxyzåäö'
    elif language=='EN':
        filename = 'words_alpha.txt'
        alphabet = 'abcdefghijklmnopqrstuvwxyz'
    with open(filename, encoding="utf-8") as f:
        all_lines = f.readlines()
    lex = []
    for line in all_lines:
        lex.append(line.rstrip())
    
    return lex, alphabet

def is_word(letters, lex):
    return letters in lex

def has_prefix(prefix, lex):
    '''
    Checks if any word in lex starts with prefix. Call get_word to get an example of a word if this function returns True.
    Arguments:
        prefix: String containing characters you wish to call.

    Returns:
       True if any word in lex starts with prefix, else False
    '''
    i = bisect.bisect_left(lex, prefix)
    if i < len(lex) and lex[i].startswith(prefix):
        return True
    return False

def get_example(letters, lex):
    '''
    Gets an example of a word that starts with letters, or empty string if none.
    '''
    i = bisect.bisect_left(lex, letters)
    if i < len(lex) and lex[i].startswith(letters):
        return lex[i]
    return ''

def take_input():
    value = input()
    while len(value) != 1 or (ord(value)<97 and value != '0' and value != '1') or (ord(value)>122 and ord(value)!=229 and ord(value)!=228 and ord(value)!=246):
        print('Illegal input')
        value = input()
    return value


### Bot definitions

In [ ]:
import random

def bogo_bot(alphabet):
    '''
    Returns random letter in alphabet
    '''
    return random.choice(alphabet)

def is_vowel(char): ## Helper function for vowel_bot
    return char in 'aouåeiyäö'
def vowel_bot(letters, alphabet):
    '''
    Returns random vowel if previous letter was consonant and vice versa
    '''
    if len(letters) == 0:
        return bogo_bot(alphabet)
    if is_vowel(letters[-1]):
        return random.choice('bcdfghjklmnpqrstvwxz')
    else:
        return random.choice('aouåeiyäö')
    
def shallow_bot(letters, lex, alphabet):
    '''
    Checks if current letter combination is a word or if it's the start of any word,
    and picks a random letter that is the continuation of a word without finishing it.
    If no such letter exists, picks a random letter according to vowel_bot()
    Returns:
        character for the action
    '''
    if is_word(letters, lex): # If letters played already forms a word, call it
        return '0'
    if not has_prefix(letters, lex): # If letters played does not form a word, call it
        return '1'
    
    # Else, find all possible next letters
    allowed_chars = []
    for letter in alphabet:
        if has_prefix(letters+letter, lex) and not is_word(letters+letter, lex):
            allowed_chars.append(letter)
    
    if len(allowed_chars) > 0:
        return random.choice(allowed_chars)
    else:
        return vowel_bot(letters, alphabet)

def recursive_length_bot_starter(prefix, lex, alphabet):
    '''
    Starter function for the recursive length bot. Uses a shallow bot if no more than 1 letters played.
    '''
    if is_word(prefix, lex):
        return '0'
    if not has_prefix(prefix, lex):
        return '1'
    
    if len(prefix) < 2:
        return shallow_bot(prefix, lex, alphabet)
    output_depth, output_string = recursive_length_bot(prefix, lex, alphabet)
    if len(output_string) == 0: # If no playable letters, use vowel_bot
        return vowel_bot(prefix, alphabet)
    return  output_string[0]

def recursive_length_bot(prefix, lex, alphabet):
    # Base case, the current letters has no continuation,
    # either due to a lack of next word or because it is a word.
    if not has_prefix(prefix, lex) or is_word(prefix, lex):
        return 0, ''
    
    # Loop over all letters in alphabet to see how far they can play
    # First dictionary: keeps the maximum depth of each letter
    # Second dictionary: keeps the letters required to reach said maximum depth
    letter_depths = {}
    required_letters = {}
    for letter in alphabet:
        depth, next_letters = recursive_length_bot(prefix+letter, lex, alphabet) # Recursive loop. Gets the maximum depth and the required letters
        letter_depths[letter] = depth + 1 # Depth increases by one
        required_letters[letter] = next_letters # Saves the output string

    # Get the maximum depth
    max_depth = max(letter_depths.values())

    # Add the maximum depth letter to the beginning of its output string
    if max_depth>1: # Only adds a letter if a playable letter exists
        best_next_letter = max(letter_depths, key=letter_depths.get)
        final_word = best_next_letter + required_letters[best_next_letter]
    else:
        final_word = ''

    return max_depth, final_word

def recursive_ratio_bot_starter(prefix, lex, alphabet):
    if is_word(prefix, lex):
        return '0'
    if not has_prefix(prefix, lex):
        return '1'
    if len(prefix) < 1:
        return shallow_bot(prefix, lex, alphabet)

    ratio, play = recursive_ratio_bot(prefix, lex, alphabet, True)
    # print(ratio)
    if play == None:
        return vowel_bot(prefix, alphabet)
    return play
    

def recursive_ratio_bot(prefix, lex, alphabet, get_play=False):
    '''
    Not perfect. It's based on maximizing the number of possible winning paths
    relative to the number of possible losing paths. One example where this fails:
    Letters currently played (bot's turn):
    'd'
    Bot suggests to play 'dr' as out of all the POSSIBLE subsequent paths, this
    has the highest win-lose ratio. An identical opponent bot's turn, plays:
    'drö'
    First bot's turn again, but can't win now, plays 'dröj'. Second bot plays
    'dröjd'. First bot loses when it plays the full word: 'dröjde'.
    '''
    # Base case, the current letters has no continuation,
    # either due to a lack of next word or because it is a word.
    if not has_prefix(prefix, lex) or is_word(prefix, lex):
        return {'w':0, 'l':0}, None # Nothing done in base case to avoid clutter
    
    # Loop over all letters in alphabet to see how far they can play
    # Dictionary: Keeps a dictionary for each letter in the alphabet.
    # The dictionaries keep track of how many routes lead to wins and loses.
    # Also: add all wins and losses to be passed back to the previous call
    w_l_ratio_dict = {}
    n_wins = 0
    n_lose = 0
    for letter in alphabet:
        ratio_dictionary = recursive_ratio_bot(prefix+letter, lex, alphabet)[0] # Recursive loop. Gets the w/l ratio of the subsequent path
        next_w = ratio_dictionary['w']
        next_l = ratio_dictionary['l']
        # Reverse the call's wins/losses
        w_l_ratio_dict[letter] = {'w' : next_l, 'l' : next_w}

        # Add wins and losses (reversed) to count
        n_wins += next_l
        n_lose += next_w

    if n_wins + n_lose == 0: # All plays lead to base case, meaning this level loses.
        n_lose = 1
    best_play = None
    if get_play: # Find the best play (only necessary on level zero, though)
        best_ratio = -1
        for letter in alphabet:
            ratio_dictionary = w_l_ratio_dict[letter]
            w = ratio_dictionary['w']
            l = ratio_dictionary['l']
            if w+l > 0: # Pick best play
                if get_play and w/(w+l) > best_ratio:
                    best_play = letter
                    best_ratio = w/(l+w)

                
    return {'w':n_wins, 'l':n_lose}, best_play

def recursive_binary_bot_starter(prefix, lex, alphabet):
    if is_word(prefix, lex):
        return '0'
    if not has_prefix(prefix, lex):
        return '1'
    if len(prefix) < 1:
        return shallow_bot(prefix, lex, alphabet)

    winning_position, play = recursive_ratio_bot(prefix, lex, alphabet, True)
    if winning_position:
        return random.choice(play)
    else:
        return recursive_ratio_bot_starter(prefix,lex,alphabet)

def recursive_binary_bot(prefix, lex, alphabet, get_play=False):
    '''
    If any subsequent letter to the ones already played in prefix is a losing position,
    this recursive algorithm marks the current prefix as a winning position as it would
    allow a bot using this algorithm to put its opponent in a losing position.
    If no such subsequent positions exist, it marks the curren position as losing.
    Only intended for 2-player games.
    Winning first letters, fyi: ['f', 'h', 'j', 'n', 'q', 'r', 'v']
    '''
    # Base case, the current letters has no continuation,
    # either due to a lack of next word or because it is a word.
    # This is a winning position, as it allows the bot to call out its opponent.
    if not has_prefix(prefix, lex) or is_word(prefix, lex):
        return True, None
    
    # Loop over all letters in alphabet to look for subsequent losing positions.
    # List 1: keeps a boolean for each letter, True for winning positions, False for losing.
    # List 2 (only relevant if get_play is True): possible next plays
    next_statuses = []
    winning_plays = []
    for letter in alphabet:
        is_next_pos_winning = recursive_binary_bot(prefix+letter, lex, alphabet)[0] # Recursive loop. Gets the status of the subsequent letter
        next_statuses.append(is_next_pos_winning)

        # If a letter is requested, add losing next positions to possible plays list
        if get_play and not is_next_pos_winning:
            winning_plays.append(letter)

    if False in next_statuses:
        is_position_winning = True
    else:
        is_position_winning = False

    if get_play:
        next_play = winning_plays
    else:
        next_play = None
    
    return is_position_winning, next_play
    



### Play game

In [ ]:
n_players = 1
n_bots = 1
language = 'SE'

lex, alphabet = get_lexicon(language)

prefix=''
curr_player = random.randint(0,n_players+n_bots-1) # If user is playing, 0 = user

while True:
    print('#############')
    print('Currently played:', flush=True)
    print(prefix, flush=True)

    if curr_player < n_players: # User(s) plays first (if not randomized)
        player = 'User ' + str(curr_player)
        print(f'{player} is up, type a letter to play, 0 to call a word, or 1 to call no word.')
        input_char = take_input()
    else: # Then bots
        player = 'Bot ' + str(curr_player-n_players)
        input_char = recursive_binary_bot_starter(prefix, lex, alphabet)

    if input_char == '0':
        print(f'{player} thinks {prefix} is a word...')
        if is_word(prefix, lex):
            print("...and it is.")
            user_victory = (curr_player < n_players)
        else:
            print('... but it is not.')
            user_victory = (curr_player >= n_players)
        break
    elif input_char == '1':
        print(f'{player} thinks {prefix} is not a start of a word...')
        word = get_example(prefix, lex)
        if word != '':
            print(f'...but it is, for example: {word}.')
            user_victory = (curr_player >= n_players)
        else:
            print('...and it is not.')
            user_victory = (curr_player < n_players)
        break

    prefix += input_char
    
    curr_player = (curr_player + 1) % (n_players+n_bots)

print('Game over')
if user_victory:
    print('You won!')
else:
    print('You lost, better luck next time!')

#############
Currently played:

#############
Currently played:
k
User 0 is up, type a letter to play, 0 to call a word, or 1 to call no word.
#############
Currently played:
kr
#############
Currently played:
kru
User 0 is up, type a letter to play, 0 to call a word, or 1 to call no word.
#############
Currently played:
kruk
#############
Currently played:
krukm
User 0 is up, type a letter to play, 0 to call a word, or 1 to call no word.
#############
Currently played:
krukma
#############
Currently played:
krukmak
User 0 is up, type a letter to play, 0 to call a word, or 1 to call no word.
#############
Currently played:
krukmaka
#############
Currently played:
krukmakar
User 0 is up, type a letter to play, 0 to call a word, or 1 to call no word.
#############
Currently played:
krukmakare
Bot 0 thinks krukmakare is a word...
...and it is.
Game over
You lost, better luck next time!
